In [7]:
!pip install PyPDF2

In [8]:
import os
pdfs = os.listdir('/kaggle/input/kenya-power-network-data/kplc_pdfs')
print(pdfs)

['01JJ9CFR02CF4Z1CN8Y3SATXKG.pdf', '01K2P78RAN74262NZS75AAQSVC.pdf', '01JC3E4SK6Z5PR3C5XKKSFVXW1.pdf', '01K2C4D1CQHWJV0PJM3ZX22S2K.pdf', '01J1FA2JXK3QNX0PM6YAP0P3T2.pdf', '01J8PS7AX192XJ1FK0KQNW0JH5.pdf', '01JPSEJ4JPEP01G1Q4VXF4NKAH.pdf', '01JRJ9TXE0PNYESAQECD70S3YN.pdf', '01JH5EB3RPVS60XPN33565DNXZ.pdf', '01K3TRXMARGN2KZ66WF6Y78TMB.pdf', '01JN333MXKH8EH77TG2KB5Z31S.pdf', '01JADBBJMQ9DD8DK6FCE0BW2K3.pdf', '01JVY1ETS6ASGH434Y6NRZREKP.pdf', '01J8PJD2N8FWYSXJ3AD11TQH8C.pdf', '01J7MY7YRCJRM5SNA8EEBXQE3S.pdf', '01JV9VDQW0A7XAWCP8VQXDTC6D.pdf', '01JTR3EEJMZGWYZXDV0TXP9XH0.pdf', '01J7185395VJDMQ36RGAQV8SJN.pdf', '01K3602Z9HTKRN3FZ15KDCZF53.pdf', '01JHQ568NP8SRE1VNM9A67NH2D.pdf', '01K4VYBGKXTQYTJGFYF94Q95YT.pdf', '01JZSGZRY4HZHA1Q9WRDV8ZT0F.pdf', '01JJV2BMR0H8MRCAD42Y1KX5FZ.pdf', '01K5H1P29K0ZC0XE3Q8RE2CWP1.pdf', '01JMHWRB64HYFS4DXNT8G2AXPS.pdf', '01JM1N87MTDJ5242CE8GGQBCRP.pdf', '01JGNPY56W5V0NXNDG3Q6JGKRK.pdf', '01J871DG040Y12R55BTGT09WYN.pdf', '01JZ7KGAR4GPMNPMRCM5P1M9ST.pdf', '01K1JKNRVT8M

In [11]:

# # Open the PDF
# with open("/kaggle/input/kenya-power-network-data/kplc_pdfs/01JT8PSACKGVT8BZXH2VA8CWXX.pdf", "rb") as f: #01J1FA2JXK3QNX0PM6YAP0P3T2.pdf
#     reader = PyPDF2.PdfReader(f)
#     print(f"Number of pages: {len(reader.pages)}")
#     text = ''
#     for j in range(len(reader.pages)):
#         # Read page
#         page = reader.pages[j]
#         text += page.extract_text()
#         # print(text)
#         #break
# print(text)

Number of pages: 2
                                                                        
                                 
Power Maintenance Notice  
Notice is hereby given under R ule 27 of the Electric Power Rules  
That the electricity supply will be interrupted as here under:   
(It is necessary to interrupt supply periodically in order to facilitate maintenance and 
upgrade of power lines to the network; to connect new customers or to replace power 
lines during road construction, etc.)  
 
NAIROBI REGION  
AREA: PART OF LORESHO  
DATE: Tuesday 06.05.2025                                           TIME: 9.00 A.M. – 5.00 P.M.  
University of Nairobi, Kaumoni, Loresho Pri & Sec, Shamba Café & adjacent 
customers . 
 
AREA: PART OF NAIROBI WEST  
DATE: Tuesday 06.05.2025                                           TIME: 9.00 A.M. - 5.00 P.M.  
Paa Crescent, ICC Church, Muthaiti Avenue, Nyayo Stadium, Nairobi West 
S/Center, Part Nairobi West, Niamey Rd, Ghandhi Ave, Kodi Rd, Summerda

In [19]:
import re
import pandas as pd
import PyPDF2



records = []
county = None
area = None
collect_places = False
places = []
files = []

for pdf in pdfs:
    # Open the PDF
    with open("/kaggle/input/kenya-power-network-data/kplc_pdfs/"+pdf, "rb") as f: #01J1FA2JXK3QNX0PM6YAP0P3T2.pdf
        reader = PyPDF2.PdfReader(f)
        print(f"Number of pages: {len(reader.pages)}")
        text = ''
        for j in range(len(reader.pages)):
            # Read page
            page = reader.pages[j]
            text += page.extract_text()
            # print(text)
            #break
        #text = text  # your notice string
        filename = pdf.split('.')[0]
        print(filename)
        files.append(filename)

        for line in text.splitlines():
            line = line.strip()
        
            # detect county
            if "PARTS OF" in line and "COUNTY" in line:
                county = line.replace("PARTS OF", "").replace("COUNTY", "").strip()
                collect_places = False
            if "NAIROBI REGION" in line:
                county = "NAIROBI"
                collect_places = False
        
            # detect area
            if line.startswith("AREA:") or line.startswith("AREA :"):
                area = line.split(":", 1)[1].strip()
                collect_places = False
        
            # detect date + time
            elif line.startswith("DATE:"):
                date = re.search(r"DATE:\s*(.*?)\s+TIME", line)
                time = re.search(r"TIME:?\s*(.*)", line)
                if date and time:
                    if places:  # flush previous record
                        records[-1]["places"] = places
                        places = []
                    records.append({
                        "county": county,
                        "area": area,
                        "date": date.group(1).strip(),
                        "time": time.group(1).strip(),
                        "places": [],
                        "file":pdf
                    })
                    collect_places = True  # start collecting places after this
        
            # collect places until the next AREA/COUNTY
            else:
                if collect_places and line and not line.startswith(("AREA", "PARTS OF", "CENTRAL", "NORTH", "WESTERN", "SOUTH", "MT.")):
                    places.extend([p.strip() for p in line.split(",") if p.strip()])
        
        # flush last block
        if records and not records[-1]["places"]:
            records[-1]["places"] = places
        
        df = pd.DataFrame(records)
        print(df.shape)


Number of pages: 2
01JJ9CFR02CF4Z1CN8Y3SATXKG
(28, 6)
Number of pages: 2
01K2P78RAN74262NZS75AAQSVC
(70, 6)
Number of pages: 1
01JC3E4SK6Z5PR3C5XKKSFVXW1
(80, 6)
Number of pages: 2
01K2C4D1CQHWJV0PJM3ZX22S2K
(118, 6)
Number of pages: 2
01J1FA2JXK3QNX0PM6YAP0P3T2
(145, 6)
Number of pages: 1
01J8PS7AX192XJ1FK0KQNW0JH5
(171, 6)
Number of pages: 2
01JPSEJ4JPEP01G1Q4VXF4NKAH
(207, 6)
Number of pages: 2
01JRJ9TXE0PNYESAQECD70S3YN
(252, 6)
Number of pages: 2
01JH5EB3RPVS60XPN33565DNXZ
(288, 6)
Number of pages: 2
01K3TRXMARGN2KZ66WF6Y78TMB
(324, 6)
Number of pages: 2
01JN333MXKH8EH77TG2KB5Z31S
(371, 6)
Number of pages: 1
01JADBBJMQ9DD8DK6FCE0BW2K3
(391, 6)
Number of pages: 2
01JVY1ETS6ASGH434Y6NRZREKP
(445, 6)
Number of pages: 1
01J8PJD2N8FWYSXJ3AD11TQH8C
(446, 6)
Number of pages: 2
01J7MY7YRCJRM5SNA8EEBXQE3S
(484, 6)
Number of pages: 2
01JV9VDQW0A7XAWCP8VQXDTC6D
(536, 6)
Number of pages: 2
01JTR3EEJMZGWYZXDV0TXP9XH0
(587, 6)
Number of pages: 2
01J7185395VJDMQ36RGAQV8SJN
(632, 6)
Number of pag

In [20]:
df.tail()

,county,area,date,time,places,file
1871,MOMBASA,MAZERAS KWA BEBORA,Friday 09.05.2025,9.00 A.M. – 5.00 P.M.,[Mwache Dam & adjacent customers.],01JT8PSACKGVT8BZXH2VA8CWXX.pdf
1872,KILIFI,MAZERAS KWA BEBORA,Tuesday 06.05.2025,9.00 A.M. – 5.00 P.M.,"[Baharini Pry, Baran Kenya Kikamabala, Kadzeng...",01JT8PSACKGVT8BZXH2VA8CWXX.pdf
1873,KILIFI,WATAMU AREA,Wednesday 07.05.2025,9.00 A.M. – 2.00 P.M.,"[Watamu Village, Watamu Police, Mawe Nzuri Res...",01JT8PSACKGVT8BZXH2VA8CWXX.pdf
1874,KWALE,SHAMU MABOKONI,Tuesday 06.05.2025,9.00 A.M. – 12.00 P.M.,"[Mabokoni, Mbuwani, Mwanjamba, Technical Unive...",01JT8PSACKGVT8BZXH2VA8CWXX.pdf
1875,KWALE,DIANI,Thursday 08.05.2025,9.00 A.M. – 3.00 P.M.,"[Carrefour S /Mkt, Swahili Beach Resort, Kaska...",01JT8PSACKGVT8BZXH2VA8CWXX.pdf


In [23]:
df['county'].unique()

array(['NAIROBI', 'NAROK', 'UASIN GISHU', 'MERU', 'ISIOLO', 'NYERI',
       'EMBU', 'KIAMBU', 'MOMBASA', 'KILIFI', 'KWALE', 'MAKUENI',
       'MACHAKOS', 'NAKURU', 'KERICHO', 'BUSIA', 'KAKAMEGA', 'KISUMU',
       'AREA:  KISII', 'MIGORI', 'KIRINYAGA', 'THARAKA NITHI', 'LAIKIPIA',
       'MURANG’A', 'KITUI', 'BOMET', 'TRANS NZOIA', 'NANDI', 'VIHIGA',
       'KISII', 'TANA RIVER', 'MAKEUNI', 'KAJIADO', 'BUNGOMA', 'HOMABAY',
       'GARISSA', 'BOMET/NAROK', 'TURKANA', 'ELGEYO MARAKWET', 'NYAMIRA',
       'NYANDADRUA', 'HOMA BAY', 'WEST POKOT', 'B UNGOMA', 'SIAYA',
       'KAKAME GA', 'KAKAMEG', 'TAITA TAVETA', 'B OMET', 'NYANDARUA',
       'TA ITA TAVETA', 'NAKURU/BARINGO', 'NAIROBI  WEST  SUB -',
       'MURANG ’A/KIAMBU', 'LAMU', 'NAIROBI SOUTH SUB -', 'BARINGO',
       'MARSABIT', 'MURANG’A/KIAMBU', 'KISU MU', 'SAMBURU'], dtype=object)

In [24]:
df.to_csv("kplc-interruptions-data.csv", index=False)